# Notebook B: Find Feasible Bounds for 13C-MFA using GSM and observed biomass yield
This Jupyter notebook focuses on using Flux Variability Analysis (FVA) to find the bounds of each reaction in a Genome-Scale Metabolic Model (GSM) that are compatible with the measured yields from growth curves. These bounds will be used to constrain the 13C-Metabolic Flux Analysis (13C-MFA) of Y. lipolytica grown on glucose, glycerol, and oleic acid.

### Objective
The objective of this analysis is to integrate 13C-MFA data with a genome-scale model to explore the metabolic capacity and flexibility under different conditions. This integration is critical for validating the GSM and providing constraints for metabolic simulations.

### Important Functions
`get_pfba_fva_df`
This function performs Flux Balance Analysis and Flux Variability Analysis given specific constraints derived from experimental data. It helps in understanding the range of possible fluxes through the metabolic network under defined conditions.

`add_mfa_bound_feasibility_column`
This function adds a column to the metabolic reaction dataframe, indicating the feasibility of MFA bounds as per the GSM data. It aids in the quick identification of reactions with feasible flux ranges and those with discrepancies.

### Structure
For each carbon source the following procedure is used to get bounds for 13C-MFA reactions
1. Get the average and standard deviation values for the yield coefficient (mmol / g dry cell weight / hr) on the carbon source
2. Calculate the cutoff for the minimum amount of biomass production using two standard deviations below the average yield
3. Run Flux Variablity Analysis (FVA) to find the flux range of each reaction given the biomass production constraint
4. Convert GSM FVA fluxes to central fluxes and add them to the central pathway dataframe
5. Plot MFA fluxes, MFA bounds, GSM fluxes, and GSM bounds

### A note about biomass cutoff calculation
Two standard deviations below the average yield was used to ensure a conservative yet realistic threshold, accounting for potential experimental errors.


### Load imports

In [1]:
import cobra
import pandas as pd
import sys
import straindesign as sd

source_dir = '../src'
sys.path.append(source_dir)
from add_gsm_bounds_from_cutoff import add_gsm_bounds_from_cutoff
from generate_flux_map import generate_flux_map

### Load the genome scale model

In [2]:
model = cobra.io.json.load_json_model('../genome_scale_models/iYLI647_corr_3.json')

### Rename some reactions to remove parentheses
This is because parentheses in reaction ids cause problems with StrainDesign

In [3]:
model.reactions.get_by_id('EX_glc(e)').id = 'EX_glc_e'
model.reactions.get_by_id('EX_glyc(e)').id = 'EX_glyc_e'
model.reactions.get_by_id('EX_ocdcea(e)').id = 'EX_ocdcea_e'
model.reactions.get_by_id('EX_h2o(e)').id = 'EX_h2o_e'
model.reactions.get_by_id('EX_h(e)').id = 'EX_h_e'
model.reactions.get_by_id('EX_nh4(e)').id = 'EX_nh4_e'
model.reactions.get_by_id('EX_o2(e)').id = 'EX_o2_e'
model.reactions.get_by_id('EX_pi(e)').id = 'EX_pi_e'
model.reactions.get_by_id('EX_so4(e)').id = 'EX_so4_e'

# print an example reaction
model.reactions.get_by_id('EX_glc_e')

Reaction identifier,EX_glc_e
Name,D Glucose exchange
Memory address,0x2d4be2740
Stoichiometry,glc_D[e] <=> D_Glucose <=>
GPR,YALI0D01111g or YALI0D18876g or YALI0D00132g or YALI0B01342g or YALI0E23287g or YALI0B00396g or...
Lower bound,-10.0
Upper bound,1000.0


### Load 13C-MFA data

In [4]:
central_rxn_df = pd.read_excel('../data/13c_mfa/INCA model 12212023_summary_GR.xlsx', sheet_name='Full MFA Data')

# calculate the number of reactions in the 13C MFA that are mapped to the GSM
mapped_rxn_df = central_rxn_df.dropna(subset = ["reaction_ids"])

print(f'There are {len(mapped_rxn_df)} reactions in the 13C MFA that are mapped to the GSM')

central_rxn_df.head()

There are 46 reactions in the 13C MFA that are mapped to the GSM


,Pathway,ID,Equation,reaction_ids,Location on map,Glucose MFA Flux,Glucose MFA LB,Glucose MFA UB,Glycerol MFA Flux,Glycerol MFA LB,Glycerol MFA UB,Oleic Acid MFA Flux,Oleic Acid MFA LB,Oleic Acid MFA UB
0,uptake,gluc uptake,Glucose + ATP -> G6P,reverse_EX_glc_e,"(-1180, 1175)",100.0000,100.0000,100.0000,NaN,NaN,NaN,NaN,NaN,NaN
1,uptake,glyc uptake,Glycerol -> GLYC,reverse_GLYCt,"(-1376, 417)",NaN,NaN,NaN,100.0000,100.0000,100.0000,NaN,NaN,NaN
2,uptake,R3.2,GLYC + ATP -> DHAP + 1.5*ATP,NaN,NaN,NaN,NaN,NaN,100.0000,100.0000,100.0000,NaN,NaN,NaN
3,uptake,OA uptake,OA + ATP -> 9*ACCOAcyt + 8*NADH + 12*ATP,OCDCEAt,"(-55, 111)",NaN,NaN,NaN,NaN,NaN,NaN,100.0000,100.0000,100.0000
4,glycolysis/gluconeogensis,R4 net,G6P <-> F6P,PGI,"(-1180, 960)",18.0496,9.7776,23.5421,-41.3947,-41.5761,-36.5906,-316.4244,-328.5227,-265.6116


### Load growth parameters 

In [5]:
# load the growth parameters from a csv
growth_parameters_df = pd.read_csv('../results/growth_parameters/growth_parameters.csv')
growth_parameters_df.set_index('Unnamed: 0', inplace=True)

growth_parameters_df

,glucose,glycerol,oleic_acid
Unnamed: 0,,,
growth_rate,0.266298,0.358041,0.260555
growth_rate_std,0.003048,0.000861,0.012321
yield_coefficient,0.110433,0.058610,0.341479
yield_coefficient_std,0.003913,0.009653,0.029548
substrate_uptake_rate,2.414362,6.266590,0.763019
substrate_uptake_rate_std,0.088704,0.959634,0.075240


# Glucose

### Get biomass cutoff from observed yield coefficient

In [6]:
# Get glucose yield coefficient and standard deviation
glucose_yield_coefficient = growth_parameters_df.loc['yield_coefficient', 'glucose']
glucose_yield_coefficient_std = growth_parameters_df.loc['yield_coefficient_std', 'glucose']

glucose_biomass_cutoff = glucose_yield_coefficient - 2 * glucose_yield_coefficient_std
print(f'glucose biomass cutoff: {glucose_biomass_cutoff} g biomass per mmol glucose')


glucose biomass cutoff: 0.102607311105205 g biomass per mmol glucose


### Add GSM bounds that are compatible with minimum biomass flux using FVA

In [7]:
central_rxn_df = add_gsm_bounds_from_cutoff(
    model = model,
    central_rxn_df = central_rxn_df,
    substrate = 'Glucose',
    uptake_reaction = 'EX_glc_e',
    biomass_cutoff = 100 * glucose_biomass_cutoff
)

central_rxn_df.head(6)

,Pathway,ID,Equation,reaction_ids,Location on map,Glucose MFA Flux,Glucose MFA LB,Glucose MFA UB,Glycerol MFA Flux,Glycerol MFA LB,Glycerol MFA UB,Oleic Acid MFA Flux,Oleic Acid MFA LB,Oleic Acid MFA UB,Glucose GSM LB,Glucose GSM UB,Glucose_mfa_bound_feasibility
0,uptake,gluc uptake,Glucose + ATP -> G6P,reverse_EX_glc_e,"(-1180, 1175)",100.0000,100.0000,100.0000,NaN,NaN,NaN,NaN,NaN,NaN,100.0,100.0,fully feasible
1,uptake,glyc uptake,Glycerol -> GLYC,reverse_GLYCt,"(-1376, 417)",NaN,NaN,NaN,100.0000,100.0000,100.0000,NaN,NaN,NaN,-26.574563,-0.0,
2,uptake,R3.2,GLYC + ATP -> DHAP + 1.5*ATP,NaN,NaN,NaN,NaN,NaN,100.0000,100.0000,100.0000,NaN,NaN,NaN,,,
3,uptake,OA uptake,OA + ATP -> 9*ACCOAcyt + 8*NADH + 12*ATP,OCDCEAt,"(-55, 111)",NaN,NaN,NaN,NaN,NaN,NaN,100.0000,100.0000,100.0000,0.0,-0.0,
4,glycolysis/gluconeogensis,R4 net,G6P <-> F6P,PGI,"(-1180, 960)",18.0496,9.7776,23.5421,-41.3947,-41.5761,-36.5906,-316.4244,-328.5227,-265.6116,-190.412199,92.008809,fully feasible
5,glycolysis/gluconeogensis,R5 net,F6P + ATP <-> FBP,PFK or reverse_FBP,"(-1180, 700)",56.7358,53.3682,59.0807,-20.6531,-20.7135,-19.0518,-145.0032,-149.0344,-128.0658,-336.832582,82.770173,fully feasible


## Glycerol

### Get biomass cutoff from observed yield coefficient

In [8]:
# Get glycerol yield coefficient
glycerol_yield_coefficient = growth_parameters_df.loc['yield_coefficient', 'glycerol']
glycerol_yield_coefficient_std = growth_parameters_df.loc['yield_coefficient_std', 'glycerol']

glycerol_biomass_cutoff = glycerol_yield_coefficient - 2 * glycerol_yield_coefficient_std
print(f'glycerol biomass cutoff: {glycerol_biomass_cutoff} g biomass per mmol glycerol')


glycerol biomass cutoff: 0.039303800604366904 g biomass per mmol glycerol


### Add GSM bounds that are compatible with minimum biomass flux using FVA

In [9]:
central_rxn_df = add_gsm_bounds_from_cutoff(
    model = model,
    central_rxn_df = central_rxn_df,
    substrate = 'Glycerol',
    uptake_reaction = 'EX_glyc_e',
    biomass_cutoff = 100 * glycerol_biomass_cutoff
)

central_rxn_df.head(6)

,Pathway,ID,Equation,reaction_ids,Location on map,Glucose MFA Flux,Glucose MFA LB,Glucose MFA UB,Glycerol MFA Flux,Glycerol MFA LB,Glycerol MFA UB,Oleic Acid MFA Flux,Oleic Acid MFA LB,Oleic Acid MFA UB,Glucose GSM LB,Glucose GSM UB,Glucose_mfa_bound_feasibility,Glycerol GSM LB,Glycerol GSM UB,Glycerol_mfa_bound_feasibility
0,uptake,gluc uptake,Glucose + ATP -> G6P,reverse_EX_glc_e,"(-1180, 1175)",100.0000,100.0000,100.0000,NaN,NaN,NaN,NaN,NaN,NaN,100.0,100.0,fully feasible,0.0,-0.0,
1,uptake,glyc uptake,Glycerol -> GLYC,reverse_GLYCt,"(-1376, 417)",NaN,NaN,NaN,100.0000,100.0000,100.0000,NaN,NaN,NaN,-26.574563,-0.0,,100.0,100.0,fully feasible
2,uptake,R3.2,GLYC + ATP -> DHAP + 1.5*ATP,NaN,NaN,NaN,NaN,NaN,100.0000,100.0000,100.0000,NaN,NaN,NaN,,,,,,
3,uptake,OA uptake,OA + ATP -> 9*ACCOAcyt + 8*NADH + 12*ATP,OCDCEAt,"(-55, 111)",NaN,NaN,NaN,NaN,NaN,NaN,100.0000,100.0000,100.0000,0.0,-0.0,,0.0,-0.0,
4,glycolysis/gluconeogensis,R4 net,G6P <-> F6P,PGI,"(-1180, 960)",18.0496,9.7776,23.5421,-41.3947,-41.5761,-36.5906,-316.4244,-328.5227,-265.6116,-190.412199,92.008809,fully feasible,-96.832349,-3.061031,fully feasible
5,glycolysis/gluconeogensis,R5 net,F6P + ATP <-> FBP,PFK or reverse_FBP,"(-1180, 700)",56.7358,53.3682,59.0807,-20.6531,-20.7135,-19.0518,-145.0032,-149.0344,-128.0658,-336.832582,82.770173,fully feasible,-168.349223,-6.599897,fully feasible


# Oleic Acid

### Get biomass cutoff from observed yield coefficient

In [10]:
# Get oleic_acid yield coefficient
oleic_acid_yield_coefficient = growth_parameters_df.loc['yield_coefficient', 'oleic_acid']
oleic_acid_yield_coefficient_std = growth_parameters_df.loc['yield_coefficient_std', 'oleic_acid']

oleic_acid_biomass_cutoff = oleic_acid_yield_coefficient - 2 * oleic_acid_yield_coefficient_std
print(f'oleic_acid biomass cutoff: {oleic_acid_biomass_cutoff} g biomass per mmol oleic_acid')


oleic_acid biomass cutoff: 0.2823833486324472 g biomass per mmol oleic_acid


### Add GSM bounds that are compatible with minimum biomass flux using FVA

In [11]:
central_rxn_df = add_gsm_bounds_from_cutoff(
    model = model,
    central_rxn_df = central_rxn_df,
    substrate = 'Oleic Acid',
    uptake_reaction = 'EX_ocdcea_e',
    biomass_cutoff = 10 * oleic_acid_biomass_cutoff
)

central_rxn_df.head(49)

,Pathway,ID,Equation,reaction_ids,Location on map,Glucose MFA Flux,Glucose MFA LB,Glucose MFA UB,Glycerol MFA Flux,Glycerol MFA LB,...,Oleic Acid MFA UB,Glucose GSM LB,Glucose GSM UB,Glucose_mfa_bound_feasibility,Glycerol GSM LB,Glycerol GSM UB,Glycerol_mfa_bound_feasibility,Oleic Acid GSM LB,Oleic Acid GSM UB,Oleic Acid_mfa_bound_feasibility
0,uptake,gluc uptake,Glucose + ATP -> G6P,reverse_EX_glc_e,"(-1180, 1175)",100.0000,100.0000,100.0000,NaN,NaN,...,NaN,100.0,100.0,fully feasible,0.0,-0.0,,0.0,-0.0,
1,uptake,glyc uptake,Glycerol -> GLYC,reverse_GLYCt,"(-1376, 417)",NaN,NaN,NaN,1.000000e+02,100.0000,...,NaN,-26.574563,-0.0,,100.0,100.0,fully feasible,-67.583966,-0.0,
2,uptake,R3.2,GLYC + ATP -> DHAP + 1.5*ATP,NaN,NaN,NaN,NaN,NaN,1.000000e+02,100.0000,...,NaN,,,,,,,,,
3,uptake,OA uptake,OA + ATP -> 9*ACCOAcyt + 8*NADH + 12*ATP,OCDCEAt,"(-55, 111)",NaN,NaN,NaN,NaN,NaN,...,100.0000,0.0,-0.0,,0.0,-0.0,,100.0,100.0,fully feasible
4,glycolysis/gluconeogensis,R4 net,G6P <-> F6P,PGI,"(-1180, 960)",18.0496,9.7776,23.5421,-4.139470e+01,-41.5761,...,-265.6116,-190.412199,92.008809,fully feasible,-96.832349,-3.061031,fully feasible,-549.764335,-24.440384,fully feasible
5,glycolysis/gluconeogensis,R5 net,F6P + ATP <-> FBP,PFK or reverse_FBP,"(-1180, 700)",56.7358,53.3682,59.0807,-2.065310e+01,-20.7135,...,-128.0658,-336.832582,82.770173,fully feasible,-168.349223,-6.599897,fully feasible,-1099.367918,-47.471995,fully feasible
6,glycolysis/gluconeogensis,R6 net,FBP <-> DHAP + GAP,FBA,"(-1184, 515)",56.7358,53.3682,59.0807,-2.065310e+01,-20.7135,...,-128.0658,-336.832582,82.770173,fully feasible,-168.349223,-6.599897,fully feasible,-1099.367918,-47.471995,fully feasible
7,glycolysis/gluconeogensis,R7 net,DHAP <-> GAP,TPI,"(-984, 273)",55.2387,51.7764,57.7436,7.870330e+01,78.6429,...,-137.4954,-12.701671,81.688971,fully feasible,59.230121,92.985948,fully feasible,-258.293933,-53.424166,fully feasible
8,glycolysis/gluconeogensis,R8 net,GAP <-> G3P + ATP + NADH,GAPD,"(-700, 250)",130.1840,124.9031,135.6598,6.793370e+01,67.8732,...,-199.2371,66.84424,163.326714,fully feasible,51.840411,85.952273,fully feasible,-306.985579,-102.115811,fully feasible
9,glycolysis/gluconeogensis,R9 net,G3P <-> PEP,ENO,"(-980, 15)",125.5023,119.9423,131.5495,6.638970e+01,66.3727,...,-202.9666,-84.753206,163.326714,fully feasible,-41.544262,85.952273,fully feasible,-613.070825,-102.115811,fully feasible


## Save the GSM bounds so they can be used to constrain the GSM

In [12]:
# save the central_rxn_df
central_rxn_df.to_csv('../results/central_fluxes/mfa_bounds_from_gsm_01192024.csv')

### Test area

In [ ]:
# SUCmit <-> FUMmit + 1.5*ATP
# SUCD2_u6m and SUCD1m

display(model.reactions.get_by_id('SUCD2_u6m'))
display(model.reactions.get_by_id('SUCD1m'))

In [ ]:
# AKGDam and AKGDbm

display(model.reactions.get_by_id('AKGDam'))
display(model.reactions.get_by_id('AKGDbm'))

In [ ]:
for r in model.metabolites.get_by_id('succoa[m]').reactions:
    display(r)

In [ ]:
for r in model.metabolites.get_by_id('cit[c]').reactions:
    if 'icit[c]' in r.reaction:
        display(r)

In [ ]:
# ACL reaction
model.reactions.get_by_id('ATPCitL')

In [ ]:
for r in model.reactions:
    if 'accoa[c]' in r.reaction:
        print(r.id, r.reaction)

In [ ]:
for r in model.reactions:
    if 'accoa[c]' in r.reaction:
        print(r.id, r.reaction)

In [ ]:
carnitine_1 = model.reactions.get_by_id('CSNAT')
print(carnitine_1.reaction)

carnitine_2 = model.reactions.get_by_id('ACRNtm')
print(carnitine_2.reaction)

carnitine_3 = model.reactions.get_by_id('CSNATifm')
print(carnitine_3.reaction)

In [ ]:
for r in model.metabolites.get_by_id('acrn[m]').reactions:
    print(r.id, r.reaction)

In [13]:
for r in model.metabolites.get_by_id('acrn[c]').reactions:
    if 'acrn[m]' in r.reaction:
        print(r.id, r.reaction)
    # print(r.id, r.reaction)

CRNCARtm acrn[c] + crn[m] --> acrn[m] + crn[c]
ACRNtm acrn[c] --> acrn[m]


In [ ]:
for r in model.metabolites.get_by_id('acrn[m]').reactions:
    if 'accoa[m]' in r.reaction:
        print(r.id, r.reaction, r.name)

In [14]:
display(model.metabolites.get_by_id('acrn[m]'))
display(model.metabolites.get_by_id('acrn[m]'))
display(model.metabolites.get_by_id('acrn[m]'))

Metabolite identifier,acrn[m]
Name,O_Acetylcarnitine
Memory address,0x1740daad0
Formula,C9H17NO4
Compartment,m
In 4 reaction(s),"CRNCARtm, ACRNtm, CSNATifm, CSNATirm"


Metabolite identifier,acrn[m]
Name,O_Acetylcarnitine
Memory address,0x1740daad0
Formula,C9H17NO4
Compartment,m
In 4 reaction(s),"CRNCARtm, ACRNtm, CSNATifm, CSNATirm"


Metabolite identifier,acrn[m]
Name,O_Acetylcarnitine
Memory address,0x1740daad0
Formula,C9H17NO4
Compartment,m
In 4 reaction(s),"CRNCARtm, ACRNtm, CSNATifm, CSNATirm"


In [ ]:
# SUCD2_u6m or SUCD1m

for r in ['SUCD2_u6m', 'SUCD1m']:
    print(model.reactions.get_by_id(r).reaction)
    

In [ ]:
for r in model.metabolites.get_by_id('succ[m]').reactions:
    print(r.id, r.reaction)

In [ ]:
# Get oleic_acid yield coefficient
oleic_acid_yield_coefficient = growth_parameters_df.loc['yield_coefficient', 'oleic_acid']
oleic_acid_yield_coefficient_std = growth_parameters_df.loc['yield_coefficient_std', 'oleic_acid']

oleic_acid_biomass_cutoff = oleic_acid_yield_coefficient - 2 * oleic_acid_yield_coefficient_std
print(f'oleic_acid biomass cutoff: {oleic_acid_biomass_cutoff} g biomass per mmol oleic_acid')


In [ ]:
import pandas as pd
from get_min_max_flux_expression_from_ids import get_min_max_flux_expression_from_ids
from add_mfa_bound_feasibility_column import add_mfa_bound_feasibility_column

substrate = 'Oleic Acid'
uptake_reaction = 'EX_ocdcea_e'
biomass_cutoff = 10 * oleic_acid_biomass_cutoff

# def add_gsm_bounds_from_cutoff(model=None, central_rxn_df=None, substrate=None, uptake_reaction=None, biomass_cutoff=None):
#     central_rxn_df = central_rxn_df.copy()

    # update the media to minimal medium with the specified sole carbon source
medium = model.medium
medium['EX_glc_e'] = 100 if substrate == 'Glucose' else 0
medium['EX_glyc_e'] = 100 if substrate == 'Glycerol' else 0
medium['EX_ocdcea_e'] = 10 if substrate == 'Oleic Acid' else 0
medium['EX_h2o_e'] = 10000
medium['EX_h_e'] = 10000
medium['EX_nh4_e'] = 10000
medium['EX_o2_e'] = 10000
medium['EX_pi_e'] = 10000
medium['EX_so4_e'] = 10000
medium['trehalose_c_tp'] = 0
model.medium = medium

# set the reaction ids for the biomass formation to glucose
if substrate == 'Glucose' or substrate == 'Glycerol':
    # define parts of the constraints string
    uptake_string = f'-{uptake_reaction} >= 100.0, -{uptake_reaction} <= 100.0'
    biomass_string = f'biomass_glucose >= {biomass_cutoff}, biomass_oil = 0, biomass_C = 0, biomass_N = 0'

    # ensure the proper biomass reaction is used in the GSM
    central_rxn_df.loc[central_rxn_df['Pathway'] == 'biomass formation', 'reaction_ids'] = 'biomass_glucose'

elif substrate == 'Oleic Acid':
    # define parts of the constraints string
    uptake_string = f'-{uptake_reaction} >= 10.0, -{uptake_reaction} <= 10.0'
    biomass_string = f'biomass_glucose = 0, biomass_oil >= {biomass_cutoff}, biomass_C = 0, biomass_N = 0'

    # ensure the proper biomass reaction is used in the GSM
    central_rxn_df.loc[central_rxn_df['Pathway'] == 'biomass formation', 'reaction_ids'] = 'biomass_oil'
else:
    raise ValueError(f'Unknown substrate: {substrate}')

# define the constraints string
constraints = f'{uptake_string}, {biomass_string}'

# remove these later
constraints = f'{constraints}, SUCOASm = 0'
constraints = f'{constraints}, SHSL1 = 0'

print(constraints)

 # run FVA to the get the pFBA flux ranges
pfba_fva_solution = sd.fva(
    model, 
    constraints=constraints,
)
print('ran pfba fva')

pfba_fva_solution

# maybe scale the fluxes to 100 uptake for oleic acid here
if substrate == 'Oleic Acid':
    pfba_fva_solution = pfba_fva_solution * 10

flux_col_label = f'{substrate}_GSM_flux'
lb_col_label = f'{substrate}_GSM_LB'
ub_col_label = f'{substrate}_GSM_UB'

# make a list of dictionaries with the reaction id, name, flux, and absolute flux
reactions = []
for reaction_id, row in pfba_fva_solution.iterrows():
    # add the reaction info to the list of dictionaries
    reactions.append({
    'reaction_id': reaction_id,
    'reaction_name': model.reactions.get_by_id(reaction_id).name,
    'full_reaction': model.reactions.get_by_id(reaction_id).reaction,
    lb_col_label: row['minimum'],
    ub_col_label: row['maximum'],
    })

# make a dataframe from the list of dictionaries
pfba_df = pd.DataFrame(reactions)

pfba_df

In [ ]:
reactions_to_show = ['SUCD2_u6m', 'SUCD1m']

pfba_df[pfba_df['reaction_id'].isin(reactions_to_show)]

In [ ]:
constraint_string = 'SUCD2_u6m + SUCD1m'

sol_min = sd.fba(model, obj=constraint_string, constraints=constraints, obj_sense='minimize')
sol_max = sd.fba(model, obj=constraint_string, constraints=constraints, obj_sense='maximize')

print(constraints)
print(f'sol min: {sol_min}')
print(f'sol max: {sol_max}')


##### Why is so much succinate being produced? 

In [ ]:
succinate_reactions = []
for r in model.metabolites.get_by_id('succ[m]').reactions:
    print(r.id, r.name, r.reaction)
    succinate_reactions.append(r.id)

succinate_reactions

In [ ]:
pfba_df[pfba_df['reaction_id'].isin(succinate_reactions)]

In [ ]:
succinate_forming_reactions = ['DICtm', 'SUCCtm', 'SUCFUMtm', 'SUCOASm']
succinate_consuming_reactions = ['SUCD1m', 'SUCD2_u6m', 'SUCOASm']

succinate_forming_string = ' + '.join(succinate_forming_reactions)
succinate_consuming_string = ' + '.join(succinate_consuming_reactions)

print(succinate_forming_string)
print(succinate_consuming_string)

In [ ]:
for r in succinate_forming_reactions:
    print(model.reactions.get_by_id(r).reaction)

In [ ]:
for r in succinate_consuming_reactions:
    print(model.reactions.get_by_id(r).reaction)

In [ ]:
# succinate production bounds
constraint_string = 'DICtm + SUCCtm + SUCFUMtm - SUCOASm'


sol_min = sd.fba(model, obj=constraint_string, constraints=constraints, obj_sense='minimize')
sol_max = sd.fba(model, obj=constraint_string, constraints=constraints, obj_sense='maximize')

print(constraints)
print(f'sol min: {sol_min}')
print(f'sol max: {sol_max}')


In [ ]:
# succinate consumption bounds
constraint_string = succinate_consuming_string


sol_min = sd.fba(model, obj=constraint_string, constraints=constraints, obj_sense='minimize')
sol_max = sd.fba(model, obj=constraint_string, constraints=constraints, obj_sense='maximize')

print(constraints)
print(f'sol min: {sol_min}')
print(f'sol max: {sol_max}')


### Why is so much succinate being produced in the cytosol that has to go into the mitochondria?

In [ ]:
succ_cytosol_reactions = []
succ_reaction_ids = []
for r in model.metabolites.get_by_id('succ[c]').reactions:
    succ_reaction_ids.append(r.id)
    succ_cytosol_reactions.append({
        'reaction_id': r.id,
        'reaction_name': r.name,
        'reaction': r.reaction,
    })
    print(r.reaction)

succ_cytosol_reactions_df = pd.DataFrame(succ_cytosol_reactions)
succ_cytosol_reactions_df

In [ ]:
pfba_df[pfba_df['reaction_id'].isin(succ_reaction_ids)]

In [ ]:
display(model.metabolites.get_by_id('sucsal[c]'))
display(model.metabolites.get_by_id('suchms[c]'))
display(model.metabolites.get_by_id('2obut[c]'))

In [ ]:
for r in model.metabolites.get_by_id('suchms[c]').reactions:
    print(r.id, r.reaction)

In [ ]:
# succinate production in cytosol bounds
constraint_string = 'ICL + SSALy'


sol_min = sd.fba(model, obj=constraint_string, constraints=constraints, obj_sense='minimize')
sol_max = sd.fba(model, obj=constraint_string, constraints=constraints, obj_sense='maximize')

print(constraints)
print(f'sol min: {sol_min}')
print(f'sol max: {sol_max}')


In [ ]:
for r in model.metabolites.get_by_id('sucsal[c]').reactions:
    print(r.id, r.reaction)

In [ ]:
icit_reactions = []
for r in model.metabolites.get_by_id('icit[c]').reactions:
    icit_reactions.append(r.id)
    print(r.id, r.reaction)

In [ ]:
pfba_df[pfba_df['reaction_id'].isin(icit_reactions)]

In [ ]:
glx_reactions = []
for r in model.metabolites.get_by_id('glx[c]').reactions:
    glx_reactions.append(r.id)
    print(r.id, r.reaction)

pfba_df[pfba_df['reaction_id'].isin(glx_reactions)]

In [ ]:
for r in model.metabolites.get_by_id('glx[x]').reactions:
    print(r.id, r.reaction)

## Old Cells

### Run FVA with yield coefficient

In [ ]:
# biomass_cutoff = 100 * glucose_biomass_cutoff
# glucose_gsm_df = get_pfba_fva_df(
#     model=model, 
#     substrate='glucose', 
#     biomass_cutoff=biomass_cutoff,
# )
# glucose_gsm_df

In [ ]:
# # look at PFK and FBP reactions
# glucose_gsm_df[glucose_gsm_df['reaction_id'].isin(['PFK', 'FBP'])]

### Add glucose FVA data to the central pathway dataframe

In [ ]:
# full_central_rxn_df = central_rxn_df.copy()

# # add pfba flux column
# full_central_rxn_df = add_flux_column_to_13c_flux_df(full_central_rxn_df, glucose_gsm_df, 'glucose_GSM_flux')
# full_central_rxn_df = add_fva_columns_to_13c_flux_df(full_central_rxn_df, glucose_gsm_df, 'glucose_GSM_LB', 'glucose_GSM_UB')
# full_central_rxn_df = add_mfa_bound_feasibility_column(full_central_rxn_df, 'glucose')

# full_central_rxn_df.head(49)

### Plot glucose MFA fluxes

In [ ]:
# glucose_mfa_map = generate_flux_map(
#     flux_df=full_central_rxn_df, 
#     flux_column='glucose_flux', 
#     title_string='Glucose 13C-MFA 2023-11-02',
#     file_name='../figures/test_flux_map.png',
# )

### Plot glucose MFA bounds (from 13C-MFA)

In [ ]:
# glucose_mfa_bounds_map = generate_flux_map(
#     flux_df=full_central_rxn_df, 
#     flux_column=['glucose_LB', 'glucose_UB'],
#     title_string='Glucose 13C-MFA Bounds 2023-11-02',
#     file_name='../figures/test_flux_map.png',
# )

### Plot glucose GSM central fluxes (biomass maximizing fluxes)

In [ ]:
# glucose_gsm_map = glucose_gsm_flux = generate_flux_map(
#     flux_df=full_central_rxn_df, 
#     flux_column='glucose_GSM_flux', 
#     title_string='Glucose pFBA GSM 2023-11-08',
#     file_name='../figures/test_flux_map.png',
# )

### Plot glucose GSM bounds for central fluxes (derived from FVA)

In [ ]:
# glucose_gsm_bounds_map = generate_flux_map(
#     flux_df=full_central_rxn_df, 
#     flux_column=['glucose_GSM_LB', 'glucose_GSM_UB'],
#     title_string=f'Glucose GSM FVA Bounds (cutoff = {biomass_cutoff:.2f}) 2023-11-08',
#     file_name='../figures/test_flux_map.png',
# )

# Glycerol

### Get biomass cutoff from observed yield coefficient

In [ ]:
# # Get glycerol yield coefficient
# glycerol_yield_coefficient = growth_parameters_df.loc['yield_coefficient', 'glycerol']
# glycerol_yield_coefficient_std = growth_parameters_df.loc['yield_coefficient_std', 'glycerol']

# glycerol_biomass_cutoff = glycerol_yield_coefficient - 2 * glycerol_yield_coefficient_std
# print(f'glycerol biomass cutoff: {glycerol_biomass_cutoff} g biomass per mmol glycerol')


### Run FVA with yield coefficient

In [ ]:
# biomass_cutoff = 100 * glycerol_biomass_cutoff
# print(biomass_cutoff)

# glycerol_gsm_df = get_pfba_fva_df(
#     model=model, 
#     substrate='glycerol', 
#     biomass_cutoff=biomass_cutoff
# )
# glycerol_gsm_df

### Add glycerol FVA data to the central pathway dataframe

In [ ]:
# # add pfba flux column
# full_central_rxn_df = add_flux_column_to_13c_flux_df(full_central_rxn_df, glycerol_gsm_df, 'glycerol_GSM_flux')
# full_central_rxn_df = add_fva_columns_to_13c_flux_df(full_central_rxn_df, glycerol_gsm_df, 'glycerol_GSM_LB', 'glycerol_GSM_UB')
# full_central_rxn_df = add_mfa_bound_feasibility_column(full_central_rxn_df, 'glycerol')

# full_central_rxn_df

### Plot glycerol MFA fluxes

In [ ]:
# glycerol_mfa_map = generate_flux_map(
#     flux_df=full_central_rxn_df, 
#     flux_column='glycerol_flux', 
#     title_string='Glycerol 13C-MFA 2023-11-02',
#     file_name='../figures/test_flux_map.png',
# )

### Plot glycerol MFA bounds (from 13C-MFA)

In [ ]:
# glycerol_mfa_bounds_map = glycerol_mfa_bounds_map = generate_flux_map(
#     flux_df=full_central_rxn_df, 
#     flux_column=['glycerol_LB', 'glycerol_UB'],
#     title_string='Glycerol 13C-MFA Bounds 2023-11-02',
#     file_name='../figures/test_flux_map.png',
# )

### Plot glycerol GSM central fluxes (biomass maximizing fluxes)

In [ ]:
# glycerol_gsm_map = generate_flux_map(
#     flux_df=full_central_rxn_df, 
#     flux_column='glycerol_GSM_flux',
#     title_string='Glycerol pFBA GSM 2023-11-08',
#     file_name='../figures/test_flux_map.png',
# )

### Plot glycerol GSM bounds for central fluxes (derived from FVA)

In [ ]:
# glycerol_gsm_bounds_map = generate_flux_map(
#     flux_df=full_central_rxn_df, 
#     flux_column=['glycerol_GSM_LB', 'glycerol_GSM_UB'],
#     title_string=f'Glycerol GSM Feasible FVA Bounds (cutoff = {biomass_cutoff:.2f}) 2023-11-08',
#     file_name='../figures/test_flux_map.png',
# )

# Oleic Acid

### Get biomass cutoff from observed yield coefficients of glucose and glycerol

In [ ]:
# # Get oleic_acid yield coefficient
# oleic_acid_yield_coefficient = growth_parameters_df.loc['yield_coefficient', 'oleic_acid']
# oleic_acid_yield_coefficient_std = growth_parameters_df.loc['yield_coefficient_std', 'oleic_acid']

# oleic_acid_biomass_cutoff = oleic_acid_yield_coefficient - 2 * oleic_acid_yield_coefficient_std
# print(f'oleic_acid biomass cutoff: {oleic_acid_biomass_cutoff} g biomass per mmol oleic_acid')


In [ ]:
# Old code to delete (gives biomass cutoff of 0.27182236847090824)

# # get the average yield coefficient normalized to number by carbon atoms (based on yield coefficients of glucose and glycerol)
# glucose_yield_coefficient_per_c = glucose_yield_coefficient / 6
# glycerol_yield_coefficient_per_c = glycerol_yield_coefficient / 3
# average_yield_coefficient_per_c = (glucose_yield_coefficient_per_c + glycerol_yield_coefficient_per_c) / 2

# # Get the standard deviation of  oleic acid yield coefficient normalized to number by carbon atoms 
# glucose_yield_coefficient_per_c_std = glucose_yield_coefficient_std / 6
# glycerol_yield_coefficient_per_c_std = glycerol_yield_coefficient_std / 3
# average_yield_coefficient_per_c_std = (glucose_yield_coefficient_per_c_std + glycerol_yield_coefficient_per_c_std) / 2

# # normalize the oleic acid yield coefficient to number by carbon atoms
# oleic_acid_yield_coefficient = average_yield_coefficient_per_c * 18
# oleic_acid_yield_coefficient_std = average_yield_coefficient_per_c_std * 18

# # calculate the biomass cutoff by subtracting 2 standard deviations from the yield coefficient
# oleic_acid_biomass_cutoff = oleic_acid_yield_coefficient - 2 * oleic_acid_yield_coefficient_std

# oleic_acid_biomass_cutoff


### Run FVA with yield coefficient

In [ ]:
# biomass_cutoff = 100 * oleic_acid_biomass_cutoff
# oleic_acid_gsm_df = get_pfba_fva_df(
#     model=model, 
#     substrate='oleic_acid', 
#     biomass_cutoff=biomass_cutoff
# )
# oleic_acid_gsm_df

### Add oleic acid FVA data to the central pathway dataframe

In [ ]:
# # add pfba flux column
# full_central_rxn_df = add_flux_column_to_13c_flux_df(full_central_rxn_df, oleic_acid_gsm_df, 'oleic_acid_GSM_flux')
# full_central_rxn_df = add_fva_columns_to_13c_flux_df(full_central_rxn_df, oleic_acid_gsm_df, 'oleic_acid_GSM_LB', 'oleic_acid_GSM_UB')
# full_central_rxn_df = add_mfa_bound_feasibility_column(full_central_rxn_df, 'oleic_acid')

# full_central_rxn_df

### Plot oleic acid MFA fluxes

In [ ]:
# oleic_acid_mfa_map = generate_flux_map(
#     flux_df=full_central_rxn_df, 
#     flux_column='oleic_acid_flux', 
#     title_string='Oleic acid 13C-MFA 2023-11-02',
#     file_name='../figures/test_flux_map.png',
# )

### Plot oleic acid MFA bounds (from 13C-MFA)

In [ ]:
# # Plot oleic acid MFA Bounds
# oleic_acid_mfa_bounds_map = generate_flux_map(
#     flux_df=full_central_rxn_df, 
#     flux_column=['oleic_acid_LB', 'oleic_acid_UB'],
#     title_string='Oleic acid 13C-MFA Bounds 2023-11-02',
#     file_name='../figures/test_flux_map.png',
# )

### Plot oleic acid GSM central fluxes (biomass maximizing fluxes)

In [ ]:
# # Plot Oleic Acid pFBA
# oleic_acid_gsm_map = generate_flux_map(
#     flux_df=full_central_rxn_df, 
#     flux_column='oleic_acid_GSM_flux', 
#     title_string='Oleic acid pFBA GSM 2023-11-08',
#     file_name='../figures/test_flux_map.png',
# )

### Plot oleic acid GSM bounds for central fluxes (derived from FVA)

In [ ]:
# oleic_acid_gsm_bounds_map = generate_flux_map(
#     flux_df=full_central_rxn_df, 
#     flux_column=['oleic_acid_GSM_LB', 'oleic_acid_GSM_UB'],
#     title_string=f'Oleic Acid GSM Feasible FVA Bounds (cutoff = {biomass_cutoff:.2f}) 2023-11-08',
#     file_name='../figures/test_flux_map.png',
# )

### Save the 13C-MFA bounds from the GSM

In [ ]:
# # reorder the columns so the the GSM flux columns are next to the 13C flux columns
# full_central_rxn_df = full_central_rxn_df[[
#     'Unnamed: 0', 'ID', 'Equation', 'reaction_ids', 'pathway', 'compartment', 
#     'glucose_flux', 'glucose_LB', 'glucose_UB', 
#     'glucose_GSM_flux', 'glucose_GSM_LB', 'glucose_GSM_UB', 'glucose_mfa_bound_feasibility',
#     'glycerol_flux', 'glycerol_LB', 'glycerol_UB',
#     'glycerol_GSM_flux', 'glycerol_GSM_LB', 'glycerol_GSM_UB', 'glycerol_mfa_bound_feasibility',
#     'oleic_acid_flux', 'oleic_acid_LB', 'oleic_acid_UB',
#     'oleic_acid_GSM_flux', 'oleic_acid_GSM_LB', 'oleic_acid_GSM_UB', 'oleic_acid_mfa_bound_feasibility'
# ]]

# full_central_rxn_df

In [ ]:
# save the full central rxn df
# full_central_rxn_df.to_csv('../results/central_fluxes/mfa_bounds_from_gsm.csv')